## Network in Network

In [1]:
!nvidia-smi

Wed Mar 16 09:50:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   34C    P0    40W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import paddle
import numpy as np

paddle.__version__, paddle.device.get_device()

('2.2.2', 'gpu:0')

### Data loading

In [3]:
import paddle.vision.transforms as T
from paddle.vision.datasets import FashionMNIST

# loading and normalization
# resize the shape to 224x224
transform = [T.Resize(size=224), T.Normalize(mean=[127.5], std=[127.5])]
transform = T.Compose(transform)

# constructing traning set and test set
fashionmnist_train = FashionMNIST(mode='train', transform=transform)
fashionmnist_test = FashionMNIST(mode='test', transform=transform)

In [4]:
# nums of train set and test set
len(fashionmnist_train), len(fashionmnist_test)

(60000, 10000)

In [5]:
# As for one sample, channel = 1, height, weight = 224
# for adapting to the input shape of NiN
fashionmnist_train[0][0].shape, fashionmnist_test[0][0].shape

((1, 224, 224), (1, 224, 224))

In [6]:
train_loader = paddle.io.DataLoader(fashionmnist_train, batch_size=256, shuffle=True)
test_loader = paddle.io.DataLoader(fashionmnist_test, batch_size=64, shuffle=False)

### Model

In [7]:
from paddle import nn

# define nin block as building blocks
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2D(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2D(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2D(out_channels, out_channels, kernel_size=1), nn.ReLU())

In [8]:
nin_net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2D(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2D(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2D(3, stride=2),
    nn.Dropout(0.5),
    # There are 10 label classes
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2D((1, 1)),
    # Transform the four-dimensional output into two-dimensional output with a
    # shape of (batch size, 10)
    nn.Flatten())

W0316 09:50:56.573233  2671 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.0, Runtime API Version: 10.1
W0316 09:50:56.576443  2671 device_context.cc:465] device: 0, cuDNN Version: 7.6.


In [9]:
# instantiation
model = paddle.Model(nin_net)

# visualization of the model workflow
model.summary((-1, 1, 224, 224)) # [N C H W]

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
     Conv2D-1        [[1, 1, 224, 224]]    [1, 96, 54, 54]        11,712     
      ReLU-1         [[1, 96, 54, 54]]     [1, 96, 54, 54]           0       
     Conv2D-2        [[1, 96, 54, 54]]     [1, 96, 54, 54]         9,312     
      ReLU-2         [[1, 96, 54, 54]]     [1, 96, 54, 54]           0       
     Conv2D-3        [[1, 96, 54, 54]]     [1, 96, 54, 54]         9,312     
      ReLU-3         [[1, 96, 54, 54]]     [1, 96, 54, 54]           0       
    MaxPool2D-1      [[1, 96, 54, 54]]     [1, 96, 26, 26]           0       
     Conv2D-4        [[1, 96, 26, 26]]     [1, 256, 26, 26]       614,656    
      ReLU-4         [[1, 256, 26, 26]]    [1, 256, 26, 26]          0       
     Conv2D-5        [[1, 256, 26, 26]]    [1, 256, 26, 26]       65,792     
      ReLU-5         [[1, 256, 26, 26]]    [1, 256, 26, 26]   

{'total_params': 1992166, 'trainable_params': 1992166}

### Training

In [10]:
# optimizer and loss
model.prepare(optimizer=paddle.optimizer.Adam(parameters=model.parameters(), learning_rate=1e-3),
              loss=nn.CrossEntropyLoss(),
              metrics=paddle.metric.Accuracy())

# training
model.fit(train_loader,
        # eval_data = test_loader
        epochs=20,
        verbose=1,
        )

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/20


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 235/235 [==============================] - loss: 0.7167 - acc: 0.4291 - 175ms/step          
Epoch 2/20
step 235/235 [==============================] - loss: 0.3616 - acc: 0.8147 - 173ms/step          
Epoch 3/20
step 235/235 [==============================] - loss: 0.4375 - acc: 0.8636 - 174ms/step          
Epoch 4/20
step 235/235 [==============================] - loss: 0.3586 - acc: 0.8817 - 174ms/step          
Epoch 5/20
step 235/235 [==============================] - loss: 0.2975 - acc: 0.8918 - 173ms/step          
Epoch 6/20
step 235/235 [==============================] - loss: 0.3766 - acc: 0.9024 - 174ms/step          
Epoch 7/20
step 235/235 [==============================] - loss: 0.2790 - acc: 0.9083 - 175ms/step          
Epoch 8/20
step 235/235 [==============================] - loss: 0.2330 - acc: 0.9143 - 173ms/step          
Epoch 9/20
step 235/235 [==============================] - loss: 0.1308 - acc: 0.9215 - 175ms/step          
Epoch 10/20
step 235/235 [====

In [11]:
model.evaluate(fashionmnist_test, verbose=1)

Eval begin...
step 10000/10000 [==============================] - loss: 0.0207 - acc: 0.9133 - 4ms/step              
Eval samples: 10000


{'loss': [0.020713678], 'acc': 0.9133}